# Chapter 13: 作用域與生命週期 | Scope and Lifetime

## Part I: 理論基礎

### 📖 章節概覽

**學習目標**：
- 理解 Python 的四種作用域（Local、Enclosing、Global、Built-in）
- 掌握 LEGB 規則的查找順序
- 學會使用 global 和 nonlocal 關鍵字
- 理解閉包（Closure）的概念與應用

**先備知識**：
- Chapter 12: 函式設計基礎

**預計時長**：60 分鐘

---

### 🔑 核心概念 1：什麼是作用域？

**作用域（Scope）**：變數名稱的可見範圍。

**生命週期（Lifetime）**：變數從創建到銷毀的時間段。

#### 為什麼需要作用域？

假設所有變數都是全域的：

In [ ]:
# 問題：沒有作用域的混亂

# 函式 1
def calculate_area():
    length = 10
    width = 5
    result = length * width
    return result

# 函式 2
def calculate_volume():
    length = 20  # 如果是全域，會覆蓋上面的 length！
    width = 10   # 如果是全域，會覆蓋上面的 width！
    height = 5
    result = length * width * height  # 覆蓋上面的 result！
    return result

# 如果沒有作用域機制：
# 1. 名稱衝突：不同函式無法使用相同變數名
# 2. 難以追蹤：無法確定變數在哪裡被修改
# 3. 記憶體浪費：所有變數永久存在

print(f"面積：{calculate_area()}")
print(f"體積：{calculate_volume()}")

**作用域的優勢**：
1. **隔離性**：每個函式有獨立的命名空間
2. **安全性**：變數的影響範圍受限
3. **記憶體效率**：函式執行完畢後，區域變數自動銷毀

### 🔑 核心概念 2：Python 的四種作用域

Python 有四種作用域，按照優先順序從高到低：

```
L (Local)      - 區域作用域：函式內部
E (Enclosing)  - 封閉作用域：外層函式
G (Global)     - 全域作用域：模組層級
B (Built-in)   - 內建作用域：Python 內建名稱
```

**LEGB 規則**：Python 按照 L → E → G → B 的順序查找變數。

---

## Part II: 實作演練

### 範例 1：區域作用域（Local Scope）

In [ ]:
def greet():
    message = "Hello"  # 區域變數
    print(message)

greet()

# 嘗試在函式外訪問區域變數
try:
    print(message)  # NameError: name 'message' is not defined
except NameError as e:
    print(f"錯誤：{e}")
    print("解釋：message 是區域變數，只存在於 greet() 內部")

**重點**：
- 區域變數只在函式內部可見
- 函式執行完畢後，區域變數自動銷毀
- 參數也是區域變數

### 範例 2：全域作用域（Global Scope）

In [ ]:
# 全域變數（模組層級）
company = "TechCorp"
employee_count = 100

def show_company_info():
    # 讀取全域變數（不需要 global 關鍵字）
    print(f"公司：{company}")
    print(f"員工數：{employee_count}")

show_company_info()

# 全域變數可在模組任何地方訪問
print(f"\n模組層級：{company}")

**重點**：
- 全域變數在整個模組中可見
- 函式內部可以**讀取**全域變數（不需要 global）
- 如果要**修改**全域變數，需要 global 關鍵字

### 範例 3：LEGB 規則演示

In [ ]:
# Global（全域）
x = "global x"

def outer():
    # Enclosing（封閉）
    x = "enclosing x"
    
    def inner():
        # Local（區域）
        x = "local x"
        print(f"inner() 內部：{x}")  # 輸出：local x
    
    inner()
    print(f"outer() 內部：{x}")  # 輸出：enclosing x

outer()
print(f"全域層級：{x}")  # 輸出：global x

**LEGB 查找順序**：
1. **L (Local)**：先找 inner() 內部的 x → 找到 "local x"
2. **E (Enclosing)**：如果沒找到，找 outer() 的 x → "enclosing x"
3. **G (Global)**：如果沒找到，找模組層級的 x → "global x"
4. **B (Built-in)**：如果沒找到，找 Python 內建名稱

### 範例 4：不同層次的變數不會互相干擾

In [ ]:
x = "global x"

def outer():
    x = "enclosing x"
    
    def inner():
        # 沒有定義 x，按 LEGB 規則查找
        print(f"inner() 內部：{x}")  # 找到 enclosing x
    
    inner()
    print(f"outer() 內部：{x}")  # enclosing x

outer()
print(f"全域層級：{x}")  # global x（未被修改）

### 範例 5：Built-in 作用域

In [ ]:
# 查看 Built-in 作用域的內容
import builtins

# Built-in 函式（最低優先權）
print("Built-in 函式範例：")
print(f"len: {len}")
print(f"print: {print}")
print(f"max: {max}")

# Built-in 常數
print(f"\nBuilt-in 常數範例：")
print(f"True: {True}")
print(f"None: {None}")

# 可以覆蓋 Built-in 名稱（但不建議）
def demo_override():
    len = "我是區域變數"  # 覆蓋 Built-in 的 len
    print(f"\n函式內部的 len：{len}")
    # print(len([1, 2, 3]))  # 錯誤！len 已不是函式

demo_override()
print(f"\n函式外部的 len：{len}")
print(f"len([1, 2, 3]) = {len([1, 2, 3])}")  # 仍可正常使用

### 範例 6：global 關鍵字的使用

In [ ]:
# 全域變數
counter = 0

def increment_wrong():
    # ❌ 錯誤：嘗試修改全域變數但沒有 global
    try:
        counter += 1  # UnboundLocalError
    except UnboundLocalError as e:
        print(f"錯誤：{e}")
        print("原因：Python 看到 counter += 1，判定 counter 是區域變數")
        print("但在賦值前嘗試讀取，導致錯誤\n")

def increment_correct():
    # ✅ 正確：使用 global 宣告
    global counter
    counter += 1
    return counter

increment_wrong()

print(f"初始值：{counter}")
print(f"第一次遞增：{increment_correct()}")
print(f"第二次遞增：{increment_correct()}")
print(f"第三次遞增：{increment_correct()}")
print(f"全域變數 counter：{counter}")

**global 使用時機**：
- 僅當需要在函式內**修改**全域變數時使用
- 只**讀取**全域變數不需要 global
- 必須在使用前宣告

### 範例 7：nonlocal 關鍵字的使用

In [ ]:
def outer():
    count = 0  # 外層函式的變數
    
    def increment_wrong():
        # ❌ 錯誤：嘗試修改外層變數但沒有 nonlocal
        try:
            count += 1  # UnboundLocalError
        except UnboundLocalError as e:
            print(f"錯誤：{e}")
            print("原因：Python 判定 count 是區域變數，但尚未賦值\n")
    
    def increment_correct():
        # ✅ 正確：使用 nonlocal 宣告
        nonlocal count
        count += 1
        return count
    
    increment_wrong()
    
    print(f"初始值：{count}")
    print(f"第一次遞增：{increment_correct()}")
    print(f"第二次遞增：{increment_correct()}")
    print(f"第三次遞增：{increment_correct()}")
    print(f"外層函式的 count：{count}")

outer()

**nonlocal vs global**：

| 關鍵字 | 作用範圍 | 使用場景 |
|:-------|:---------|:---------|
| `global` | 模組層級的全域變數 | 修改全域變數 |
| `nonlocal` | 外層函式的變數（不包括全域） | 修改外層函式變數 |

### 範例 8：閉包（Closure）的基本概念

In [ ]:
def make_counter():
    """創建一個計數器閉包"""
    count = 0  # 自由變數（free variable）
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    return increment  # 回傳內層函式

# 創建兩個獨立的計數器
counter1 = make_counter()
counter2 = make_counter()

print("counter1:")
print(counter1())  # 1
print(counter1())  # 2
print(counter1())  # 3

print("\ncounter2:")
print(counter2())  # 1（獨立的計數器）
print(counter2())  # 2

print("\ncounter1:")
print(counter1())  # 4（繼續計數）

**閉包的三個條件**：
1. **巢狀函式**：有內層函式
2. **引用外層變數**：內層函式使用外層函式的變數
3. **回傳內層函式**：外層函式回傳內層函式

**閉包的優勢**：
- **封裝性**：狀態被封裝在函式內，外部無法直接訪問
- **獨立性**：每個閉包都有自己的狀態
- **持久性**：即使外層函式執行完畢，狀態仍保留

### 範例 9：閉包的實際應用 - 工廠函式

In [ ]:
def make_multiplier(n):
    """創建一個乘法函式"""
    def multiplier(x):
        return x * n
    return multiplier

# 創建不同倍數的函式
times2 = make_multiplier(2)
times3 = make_multiplier(3)
times10 = make_multiplier(10)

print("times2:")
print(f"times2(5) = {times2(5)}")    # 10
print(f"times2(10) = {times2(10)}")  # 20

print("\ntimes3:")
print(f"times3(5) = {times3(5)}")    # 15
print(f"times3(10) = {times3(10)}")  # 30

print("\ntimes10:")
print(f"times10(5) = {times10(5)}")    # 50
print(f"times10(10) = {times10(10)}")  # 100

### 範例 10：常見陷阱 - UnboundLocalError

In [ ]:
x = 10

def demo_error():
    """演示 UnboundLocalError"""
    try:
        print(x)  # UnboundLocalError!
        x = 20    # Python 看到這行，判定 x 是區域變數
    except UnboundLocalError as e:
        print(f"錯誤：{e}")
        print("\n原因分析：")
        print("1. Python 在編譯時掃描函式")
        print("2. 看到 'x = 20'，判定 x 是區域變數")
        print("3. 在 print(x) 時，x 尚未賦值")
        print("4. 導致 UnboundLocalError")

print("測試 1：演示錯誤")
demo_error()

print("\n" + "="*50)

# 解決方法 1：只讀取（不修改）
def solution1():
    print(f"讀取全域變數 x：{x}")

print("\n測試 2：解決方法 1（只讀取）")
solution1()

print("\n" + "="*50)

# 解決方法 2：使用 global
def solution2():
    global x
    print(f"讀取全域變數 x：{x}")
    x = 20
    print(f"修改後的 x：{x}")

print("\n測試 3：解決方法 2（使用 global）")
solution2()
print(f"全域變數 x：{x}")

print("\n" + "="*50)

# 解決方法 3：使用不同的變數名
x = 10  # 恢復原值

def solution3():
    print(f"讀取全域變數 x：{x}")
    local_x = x + 10
    print(f"區域變數 local_x：{local_x}")

print("\n測試 4：解決方法 3（使用不同變數名）")
solution3()
print(f"全域變數 x：{x}（未被修改）")

---

## Part III: 本章總結

### 📝 知識回顧

#### 1. 四種作用域

| 作用域 | 說明 | 範例 |
|:-------|:-----|:-----|
| **Local** | 函式內部定義的變數 | `def func(): x = 1` |
| **Enclosing** | 外層函式的作用域 | 巢狀函式中的外層變數 |
| **Global** | 模組層級的變數 | 檔案開頭定義的變數 |
| **Built-in** | Python 內建名稱 | `len`, `print`, `True` |

#### 2. LEGB 規則

```
查找順序：Local → Enclosing → Global → Built-in
特點：由內而外，找到後立即停止
```

#### 3. global vs nonlocal

```python
# global：修改全域變數
x = 10
def func():
    global x
    x = 20

# nonlocal：修改外層函式變數
def outer():
    x = 10
    def inner():
        nonlocal x
        x = 20
```

#### 4. 閉包的三個條件

1. 有巢狀函式
2. 內層函式引用外層變數
3. 外層函式回傳內層函式

---

### ⚠️ 常見誤區

#### 誤區 1：認為讀取全域變數也需要 global

```python
# ❌ 錯誤認知
x = 10
def func():
    global x  # 只讀取不需要 global
    print(x)

# ✅ 正確
x = 10
def func():
    print(x)  # 直接讀取即可
```

#### 誤區 2：混淆 global 和 nonlocal

```python
# global：針對模組層級
# nonlocal：針對外層函式（不包括全域）

x = "global"
def outer():
    x = "enclosing"
    def inner():
        nonlocal x  # 修改 outer 的 x，不是全域的 x
        x = "local"
```

#### 誤區 3：不理解 UnboundLocalError 的成因

```python
# Python 在編譯時決定變數的作用域
# 看到賦值語句（x = ...）就判定 x 是區域變數
```

---

### ✅ 自我檢核

完成本講義後，您應該能夠：

- [ ] 識別變數屬於哪種作用域（L/E/G/B）
- [ ] 使用 LEGB 規則預測變數查找結果
- [ ] 正確使用 global 關鍵字修改全域變數
- [ ] 正確使用 nonlocal 關鍵字修改外層變數
- [ ] 理解閉包的概念與應用
- [ ] 診斷並修正 UnboundLocalError
- [ ] 實作簡單的閉包函式（如計數器）

---

### 📚 延伸閱讀

1. **Python 官方文件**：
   - [Python Scopes and Namespaces](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)

2. **推薦文章**：
   - [Real Python: Python Scope & the LEGB Rule](https://realpython.com/python-scope-legb-rule/)

3. **下一步**：
   - 完成 `02-worked-examples.ipynb` 的詳解範例
   - 完成 `03-practice.ipynb` 的課堂練習

---

**學習提醒**：作用域是 Python 的核心機制，請務必透過大量練習建立扎實基礎！